In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import statistics
import sklearn.metrics as metrics
# import tensorflow as tf
from sklearn.metrics import f1_score

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import fbeta_score
from sklearn.feature_selection import RFECV
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler

# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from sklearn.ensemble import RandomForestClassifier
# from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, roc_curve, auc, roc_auc_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

In [2]:
train = pd.read_csv("../new_train.csv")
test = pd.read_csv("../new_test.csv")

In [3]:
train

,Unnamed: 0,bidder_id,payment_account,address,outcome,auction,merchandise,device,time,country,...,max_url_per_auction,min_url_per_auction,std_url_per_auction,total_no_of_participated_auctions,no_of_auction_exceeds_threshold,percentage_of_auctions_above_threshold,total_no_of_bidded_category,no_of_merchandise_exceeds_threshold,percentage_of_merchandise_above_threshold,on_url_that_has_a_bot_mean
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,18.0,1.0,14.0,24.0,6.0,...,1.0,1.0,0.000000,18.0,0.0,0.000000,1.0,0.0,0.0,1.000000
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,1.0,1.0,2.0,3.0,1.0,...,2.0,2.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,0.500000
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,1.0,2.0,4.0,1.0,...,1.0,1.0,0.000000,4.0,0.0,0.000000,1.0,0.0,0.0,0.500000
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,1.000000
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,23.0,1.0,53.0,155.0,2.0,...,21.0,1.0,5.644263,23.0,1.0,0.043478,1.0,0.0,0.0,0.010989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008,2008,369515b3af4f8ca582f90271d30b14b6r52aw,a1f85275793c4a782f0a668711f41b927ivc9,e6882cf204a9482edd042b6e31791dfctxzx8,0.0,25.0,1.0,4.0,33.0,4.0,...,1.0,1.0,0.000000,25.0,1.0,0.040000,1.0,0.0,0.0,0.500000
2009,2009,f939c17ffc7c39ac9b35b69e5e75179fv9pe2,a3d2de7675556553a5f08e4c88d2c2286s1m2,b9b03d5a127eb07aeb9163cdcf524e1344ac9,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,0.000000
2010,2010,c806dbb2decba0ed3c4ff5e2e60a74c2wjvbl,a3d2de7675556553a5f08e4c88d2c22856leq,d02c2b288b8aabd79ff47118aff41a2dqwzwc,0.0,1.0,1.0,2.0,2.0,1.0,...,1.0,1.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,0.000000
2011,2011,0381a69b7a061e9ace2798fd48f1f537mgq57,fd87037ce0304077079c749f420f0b4c54uo0,f030a221726fbcdfc4dc7dfd1b381a112hieq,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,0.000000


In [4]:
train.drop(train.filter(regex="Unname"),axis=1, inplace=True)
test.drop(test.filter(regex="Unname"),axis=1, inplace=True)

In [5]:
X = train.drop(columns=['bidder_id', 'payment_account', 'address', 'outcome','merchandise']) 
y = train['outcome']
X_test_original = test.drop(columns=['bidder_id', 'payment_account', 'address', 'merchandise'])

### Scaling the training and test dataset

In [6]:
X.shape

(2013, 52)

In [7]:
X.columns

Index(['auction', 'device', 'time', 'country', 'ip', 'url', 'num_bids',
       'num_first_bids', 'num_last_bids', 'time_to_bid', 'inst_resp',
       'perc_inst_resp', 'auto parts', 'books and music', 'clothing',
       'computers', 'furniture', 'home goods', 'jewelry', 'mobile',
       'office equipment', 'sporting goods', 'num_bids_per_auction',
       'num_bids_per_device', 'num_bids_per_country', 'num_bids_per_ip',
       'on_ip_that_has_a_bot_mean', 'ip_entropy', 'url_entropy',
       'mean_country_per_auction', 'max_country_per_auction',
       'min_country_per_auction', 'std_country_per_auction',
       'mean_devices_per_auction', 'max_devices_per_auction',
       'min_devices_per_auction', 'std_devices_per_auction',
       'mean_ip_per_auction', 'max_ip_per_auction', 'min_ip_per_auction',
       'std_ip_per_auction', 'mean_url_per_auction', 'max_url_per_auction',
       'min_url_per_auction', 'std_url_per_auction',
       'total_no_of_participated_auctions', 'no_of_auction_excee

In [8]:
scaler = StandardScaler()
# scaled_features = data.copy()
col_names = ['auction', 'device', 'time', 'country', 'ip', 'url', 'num_bids',
       'num_first_bids', 'num_last_bids', 'time_to_bid', 'inst_resp',
       'perc_inst_resp', 'num_bids_per_auction',
       'num_bids_per_device', 'num_bids_per_country', 'num_bids_per_ip',
       'on_ip_that_has_a_bot_mean', 'ip_entropy', 'url_entropy',
       'mean_country_per_auction', 'max_country_per_auction',
       'min_country_per_auction', 'std_country_per_auction',
       'mean_devices_per_auction', 'max_devices_per_auction',
       'min_devices_per_auction', 'std_devices_per_auction',
       'mean_ip_per_auction', 'max_ip_per_auction', 'min_ip_per_auction',
       'std_ip_per_auction', 'mean_url_per_auction', 'max_url_per_auction',
       'min_url_per_auction', 'std_url_per_auction',
       'total_no_of_participated_auctions', 'no_of_auction_exceeds_threshold',
       'percentage_of_auctions_above_threshold', 'total_no_of_bidded_category',
       'no_of_merchandise_exceeds_threshold',
       'percentage_of_merchandise_above_threshold',
       'on_url_that_has_a_bot_mean']

train_features = X[col_names]
scaler = StandardScaler().fit(train_features.values)
train_features = scaler.transform(train_features.values)
X[col_names] = train_features

test_features = X_test_original[col_names]
scaler_test = StandardScaler().fit(test_features.values)
test_features = scaler_test.transform(test_features.values)
X_test_original[col_names] = test_features

In [9]:
selected_features = ['num_bids_per_ip', 'num_bids_per_auction', 'num_bids_per_country', 'mean_ip_per_auction', 'perc_inst_resp', 'time', 'mean_url_per_auction', 'inst_resp', 'num_bids', 'time_to_bid', 'num_last_bids', 'on_url_that_has_a_bot_mean', 'device', 'percentage_of_auctions_above_threshold', 'num_bids_per_device', 'ip_entropy', 'ip', 'max_country_per_auction', 'max_ip_per_auction', 'mean_devices_per_auction', 'url_entropy', 'std_ip_per_auction', 'url', 'max_devices_per_auction', 'no_of_auction_exceeds_threshold', 'std_devices_per_auction', 'std_url_per_auction', 'country', 'num_first_bids', 'mobile', 'max_url_per_auction', 'computers', 'percentage_of_merchandise_above_threshold', 'auction', 'std_country_per_auction', 'books and music', 'total_no_of_participated_auctions', 'jewelry', 'no_of_merchandise_exceeds_threshold', 'on_ip_that_has_a_bot_mean']
X = X[selected_features]
X_test_original = X_test_original[selected_features]

In [10]:
len(selected_features)

40

### Helper Functions

In [11]:
def evaluate(model, X_train, y_train, X_test, y_test):
    print("TRAIN")
    train_predictions = model.predict_proba(X_train)
    train_pred = model.predict(X_train)
        
    train_accuracy = accuracy_score(y_train, train_pred)
    train_auc_roc_score = roc_auc_score(y_train,train_predictions[:,1])
    train_fbeta = fbeta_score(y_train, train_pred, average='binary', beta=2.0)
    
    print("Classification report")
    print(classification_report(y_train, train_pred, digits = 4))
    
    print("FBeta Score")
    print(fbeta_score(y_train, train_pred, average='binary', beta=2.0))
    
    print('Model Performance')
    print('Accuracy = {:0.4f}%.'.format(train_accuracy))
    print('AUC ROC = {:0.4f}%.'.format(train_auc_roc_score))
    print("*" * 100)
    
    print("TEST")
    
    test_predictions = model.predict_proba(X_test)
    test_pred = model.predict(X_test)
        
    test_accuracy = accuracy_score(y_test, test_pred)
    test_auc_roc_score = roc_auc_score(y_test,test_predictions[:,1])
    test_fbeta = fbeta_score(y_test, test_pred, average='binary', beta=2.0)
    
    print("Classification report")
    print(classification_report(y_test, test_pred, digits = 4))
    
    print("FBeta Score")
    print(fbeta_score(y_test, test_pred, average='binary', beta=2.0))
    
    print('Model Performance')
    print('Accuracy = {:0.4f}%.'.format(test_accuracy))
    print('AUC ROC = {:0.4f}%.'.format(test_auc_roc_score))
    print("*" * 100)
    
    return [train_accuracy, train_auc_roc_score, train_fbeta, test_accuracy, test_auc_roc_score, test_fbeta]

In [12]:
def process_results(arr):
    train_accuracy = []
    train_auc_roc_score = [] 
    train_fbeta = []
    test_accuracy = []
    test_auc_roc_score = []
    test_fbeta = []
    
    for item in arr:
        train_accuracy.append(item[0])
        train_auc_roc_score.append(item[1])
        train_fbeta.append(item[2])
        test_accuracy.append(item[3])
        test_auc_roc_score.append(item[4])
        test_fbeta.append(item[5])
    
    mean_accuracy = np.array(train_accuracy).mean()
    mean_train_auc_roc_score = np.array(train_auc_roc_score).mean()
    mean_train_fbeta = np.array(train_fbeta).mean()
    mean_test_accuracy = np.array(test_accuracy).mean()
    mean_test_auc_roc_score = np.array(test_auc_roc_score).mean()
    mean_test_fbeta = np.array(test_fbeta).mean()
    
    print("final train accuracy: " + str(mean_accuracy))
    print("final train AUC: " + str(mean_train_auc_roc_score))
    print("final train fbeta: " + str(mean_train_fbeta))
    print("final test accuracy: " + str(mean_test_accuracy))
    print("final test AUC: " + str(mean_test_auc_roc_score))
    print("final test fbeta: " + str(mean_test_fbeta))

### SKFold + SMOTE (Base Model)

In [13]:
def create_model(learn_rate=0.01, momentum=0):
    # create model
    model = Sequential()
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
counter = 1
dt_result = []
rf_result = []

over = SMOTE(sampling_strategy=0.2, random_state = 42)
under = RandomUnderSampler(sampling_strategy=0.4, random_state = 42)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)

for train_index, test_index in skf.split(X, y):
    print("Fold number " + str(counter))
    counter += 1
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    X_train, y_train = pipeline.fit_resample(X_train, y_train)
    
    # decision tree
    print("DECISION TREE")
    dt = DecisionTreeClassifier(random_state=42)
    dt.fit(X_train, y_train)
    dt_result.append(evaluate(dt, X_train, y_train, X_test, y_test))
    
    # random forest
    print("RANDOM FOREST")
    rf = RandomForestClassifier(random_state=42)
    rf.fit(X_train, y_train)
    rf_result.append(evaluate(rf,X_train,y_train,X_test,y_test))

Fold number 1
DECISION TREE
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     1.0000    1.0000    1.0000       762
         1.0     1.0000    1.0000    1.0000       305

    accuracy                         1.0000      1067
   macro avg     1.0000    1.0000    1.0000      1067
weighted avg     1.0000    1.0000    1.0000      1067

FBeta Score
1.0
Model Performance
Accuracy = 1.0000%.
AUC ROC = 1.0000%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9775    0.9110    0.9431       382
         1.0     0.2766    0.6190    0.3824        21

    accuracy                         0.8958       403
   macro avg     0.6271    0.7650    0.6627       403
weighted avg     0.9410    0.8958    0.9139       403

FBeta Score
0.4961832061068703
Model Performance
Accuracy = 0.8958%.
AUC ROC = 0.7650%.
*

Classification report
              precision    recall  f1-score   support

         0.0     0.9811    0.9529    0.9668       382
         1.0     0.4194    0.6500    0.5098        20

    accuracy                         0.9378       402
   macro avg     0.7002    0.8014    0.7383       402
weighted avg     0.9532    0.9378    0.9441       402

FBeta Score
0.5855855855855855
Model Performance
Accuracy = 0.9378%.
AUC ROC = 0.9058%.
****************************************************************************************************


In [14]:
process_results(dt_result)

final train accuracy: 1.0
final train AUC: 1.0
final train fbeta: 1.0
final test accuracy: 0.8981667345653864
final test AUC: 0.7085190725504862
final test fbeta: 0.4100391015206563


In [15]:
process_results(rf_result)

final train accuracy: 0.9998125585754452
final train AUC: 1.0
final train fbeta: 0.999475065616798
final test accuracy: 0.9349233978988434
final test AUC: 0.9135994764397906
final test fbeta: 0.535766388015879


### SKFold + SMOTE (Random Search)

In [16]:
def random_search_dt(X, y):
    print("RANDOM SEARCH DECISION TREE EXPERIMENT")
    criterion = ['gini', 'entropy']
    max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
    max_depth.append(None)
    min_samples_split = [1,2,5,8]
    min_samples_leaf = [1,11,21,31]
    max_features = [5,10,15,25]
    min_impurity_decrease = [0.00005,0.0005,0.005,0.05]

    dt_random_grid = {'criterion': criterion,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'max_features': max_features,
                   'min_samples_leaf': min_samples_leaf, 
                   'min_impurity_decrease': min_impurity_decrease}
    
    
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    over = SMOTE(sampling_strategy=0.2, random_state = 42)
    under = RandomUnderSampler(sampling_strategy=0.4, random_state = 42)
    steps = [('o', over), ('u', under)]
    pipeline = Pipeline(steps=steps)
    X_train, y_train = pipeline.fit_resample(X_train, y_train)
    
    
    dt = DecisionTreeClassifier(random_state=42)
    dt_random = RandomizedSearchCV(estimator = dt, param_distributions = dt_random_grid, n_iter = 100, cv = skf, 
                               verbose=2, random_state=42, n_jobs = -1, scoring='roc_auc')
    dt_random.fit(X_train, y_train)
    
    dt_best_random = dt_random.best_estimator_
    dt_random_accuracy = evaluate(dt_best_random, X_train, y_train, X_test, y_test)
    print("Best Random Search Param for DT")
    print(dt_random.best_params_)
    
def random_search_rf(X, y):
    print("RANDOM SEARCH RANDOM FOREST EXPERIMENT")
    # parameters for Random Forest
    n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
    max_features = ['auto', 'sqrt',5,10,25]
    max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
    max_depth.append(None)
    min_samples_split = [2, 5, 10]
    min_samples_leaf = [1, 2, 4,8,10]
    bootstrap = [True, False]

    # Create the random grid
    rf_random_grid = {'n_estimators': n_estimators,
                   'max_features': max_features,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf,
                   'bootstrap': bootstrap}
    
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    over = SMOTE(sampling_strategy=0.2, random_state = 42)
    under = RandomUnderSampler(sampling_strategy=0.4, random_state = 42)
    steps = [('o', over), ('u', under)]
    pipeline = Pipeline(steps=steps)
    X_train, y_train = pipeline.fit_resample(X_train, y_train)
    
    rf = RandomForestClassifier(random_state=42)
    rf_random = RandomizedSearchCV(estimator = rf, param_distributions = rf_random_grid, n_iter = 100, cv = skf, verbose=2, 
                               random_state=42, n_jobs = -1, scoring='roc_auc')
    rf_random.fit(X_train, y_train)
    
    rf_best_random = rf_random.best_estimator_
    rf_random_accuracy = evaluate(rf_best_random, X_train, y_train, X_test, y_test)
    print("Best Random Search Param for RF")
    print(rf_random.best_params_)
    

In [17]:
random_search_dt(X, y)

RANDOM SEARCH DECISION TREE EXPERIMENT
Fitting 5 folds for each of 100 candidates, totalling 500 fits
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.9280    0.9113    0.9196       665
         1.0     0.7878    0.8233    0.8051       266

    accuracy                         0.8861       931
   macro avg     0.8579    0.8673    0.8624       931
weighted avg     0.8880    0.8861    0.8869       931

FBeta Score
0.815946348733234
Model Performance
Accuracy = 0.8861%.
AUC ROC = 0.9562%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9846    0.8839    0.9315       577
         1.0     0.2209    0.7037    0.3363        27

    accuracy                         0.8758       604
   macro avg     0.6027    0.7938    0.6339       604
weighted avg     0.9504    0.8758    0.9049       604

F

/opt/homebrew/Caskroom/miniforge/base/envs/.venv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
155 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
155 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/.venv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/homebrew/Caskroom/miniforge/base/envs/.venv/lib/python3.9/site-packages/sklearn/tree/_classes.py", line 937, in fit
    super().fit(
  File "/opt/homebrew/Caskroom/miniforge/base/envs/.venv/lib/python3.9/site-packages/sklearn/

In [18]:
random_search_rf(X, y)

RANDOM SEARCH RANDOM FOREST EXPERIMENT
Fitting 5 folds for each of 100 candidates, totalling 500 fits
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.9985    1.0000    0.9992       665
         1.0     1.0000    0.9962    0.9981       266

    accuracy                         0.9989       931
   macro avg     0.9992    0.9981    0.9987       931
weighted avg     0.9989    0.9989    0.9989       931

FBeta Score
0.9969902182091799
Model Performance
Accuracy = 0.9989%.
AUC ROC = 1.0000%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9774    0.9757    0.9766       577
         1.0     0.5000    0.5185    0.5091        27

    accuracy                         0.9553       604
   macro avg     0.7387    0.7471    0.7428       604
weighted avg     0.9561    0.9553    0.9557       604



[CV] END criterion=entropy, max_depth=60, max_features=15, min_impurity_decrease=5e-05, min_samples_leaf=21, min_samples_split=2; total time=   0.0s
[CV] END criterion=gini, max_depth=None, max_features=10, min_impurity_decrease=0.005, min_samples_leaf=31, min_samples_split=8; total time=   0.0s
[CV] END criterion=entropy, max_depth=10, max_features=10, min_impurity_decrease=5e-05, min_samples_leaf=11, min_samples_split=5; total time=   0.0s
[CV] END criterion=entropy, max_depth=20, max_features=15, min_impurity_decrease=0.005, min_samples_leaf=1, min_samples_split=8; total time=   0.0s
[CV] END criterion=gini, max_depth=30, max_features=5, min_impurity_decrease=0.0005, min_samples_leaf=11, min_samples_split=8; total time=   0.0s
[CV] END criterion=gini, max_depth=60, max_features=10, min_impurity_decrease=5e-05, min_samples_leaf=31, min_samples_split=2; total time=   0.0s
[CV] END criterion=gini, max_depth=None, max_features=5, min_impurity_decrease=0.005, min_samples_leaf=31, min_sam

[CV] END criterion=entropy, max_depth=50, max_features=5, min_impurity_decrease=0.05, min_samples_leaf=31, min_samples_split=8; total time=   0.0s
[CV] END criterion=gini, max_depth=None, max_features=10, min_impurity_decrease=0.005, min_samples_leaf=31, min_samples_split=8; total time=   0.0s
[CV] END criterion=gini, max_depth=None, max_features=10, min_impurity_decrease=0.005, min_samples_leaf=31, min_samples_split=8; total time=   0.0s
[CV] END criterion=entropy, max_depth=20, max_features=15, min_impurity_decrease=0.005, min_samples_leaf=1, min_samples_split=8; total time=   0.0s
[CV] END criterion=gini, max_depth=None, max_features=5, min_impurity_decrease=0.0005, min_samples_leaf=11, min_samples_split=1; total time=   0.0s
[CV] END criterion=gini, max_depth=None, max_features=10, min_impurity_decrease=0.0005, min_samples_leaf=11, min_samples_split=5; total time=   0.0s
[CV] END criterion=gini, max_depth=30, max_features=5, min_impurity_decrease=0.005, min_samples_leaf=1, min_samp

[CV] END criterion=entropy, max_depth=60, max_features=15, min_impurity_decrease=5e-05, min_samples_leaf=21, min_samples_split=2; total time=   0.0s
[CV] END criterion=gini, max_depth=110, max_features=25, min_impurity_decrease=0.0005, min_samples_leaf=31, min_samples_split=1; total time=   0.0s
[CV] END criterion=gini, max_depth=110, max_features=25, min_impurity_decrease=0.0005, min_samples_leaf=31, min_samples_split=1; total time=   0.0s
[CV] END criterion=gini, max_depth=30, max_features=5, min_impurity_decrease=0.005, min_samples_leaf=1, min_samples_split=1; total time=   0.0s
[CV] END criterion=gini, max_depth=30, max_features=5, min_impurity_decrease=0.005, min_samples_leaf=1, min_samples_split=1; total time=   0.0s
[CV] END criterion=entropy, max_depth=40, max_features=15, min_impurity_decrease=5e-05, min_samples_leaf=21, min_samples_split=8; total time=   0.0s
[CV] END criterion=gini, max_depth=90, max_features=15, min_impurity_decrease=0.0005, min_samples_leaf=11, min_samples

[CV] END criterion=entropy, max_depth=50, max_features=5, min_impurity_decrease=0.05, min_samples_leaf=31, min_samples_split=8; total time=   0.0s
[CV] END criterion=entropy, max_depth=10, max_features=10, min_impurity_decrease=5e-05, min_samples_leaf=11, min_samples_split=5; total time=   0.0s
[CV] END criterion=entropy, max_depth=40, max_features=15, min_impurity_decrease=5e-05, min_samples_leaf=21, min_samples_split=8; total time=   0.0s
[CV] END criterion=entropy, max_depth=40, max_features=15, min_impurity_decrease=5e-05, min_samples_leaf=21, min_samples_split=8; total time=   0.0s
[CV] END criterion=gini, max_depth=None, max_features=5, min_impurity_decrease=0.005, min_samples_leaf=31, min_samples_split=1; total time=   0.0s
[CV] END criterion=gini, max_depth=None, max_features=5, min_impurity_decrease=0.005, min_samples_leaf=31, min_samples_split=1; total time=   0.0s
[CV] END criterion=entropy, max_depth=40, max_features=10, min_impurity_decrease=0.05, min_samples_leaf=21, min_

[CV] END criterion=entropy, max_depth=60, max_features=15, min_impurity_decrease=5e-05, min_samples_leaf=21, min_samples_split=2; total time=   0.0s
[CV] END criterion=entropy, max_depth=10, max_features=10, min_impurity_decrease=5e-05, min_samples_leaf=11, min_samples_split=5; total time=   0.0s
[CV] END criterion=entropy, max_depth=20, max_features=15, min_impurity_decrease=0.005, min_samples_leaf=1, min_samples_split=8; total time=   0.0s
[CV] END criterion=gini, max_depth=30, max_features=5, min_impurity_decrease=0.0005, min_samples_leaf=11, min_samples_split=8; total time=   0.0s
[CV] END criterion=gini, max_depth=None, max_features=5, min_impurity_decrease=0.0005, min_samples_leaf=11, min_samples_split=1; total time=   0.0s
[CV] END criterion=gini, max_depth=None, max_features=10, min_impurity_decrease=0.0005, min_samples_leaf=11, min_samples_split=5; total time=   0.0s
[CV] END criterion=gini, max_depth=None, max_features=10, min_impurity_decrease=0.0005, min_samples_leaf=11, mi


*****************************************************************************************
## Best Random Search Param for DT
{'min_samples_split': 2, 'min_samples_leaf': 31, 'min_impurity_decrease': 0.0005, 'max_features': 15, 'max_depth': None, 'criterion': 'entropy'}

## Best Random Search Param for RF
{'n_estimators': 1200, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 10, 'max_depth': 30, 'bootstrap': False}

### SKFold + SMOTE (Grid Search)

In [19]:
def grid_search_dt(X, y):
    print("GRID SEARCH DECISION TREE EXPERIMENT")
    dt_grid = {'criterion': ['entropy'],
        'max_depth': [None],
        'max_features': [14,15,16],
        'min_impurity_decrease': [0.0005,0.0004,0.0006],
        'min_samples_leaf': [30,31,32],
        'min_samples_split': [1,2,3]
    }
    
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    over = SMOTE(sampling_strategy=0.2, random_state = 42)
    under = RandomUnderSampler(sampling_strategy=0.4, random_state = 42)
    steps = [('o', over), ('u', under)]
    pipeline = Pipeline(steps=steps)
    X_train, y_train = pipeline.fit_resample(X_train, y_train)
    
    dt = DecisionTreeClassifier(random_state=42)
    dt_grid = GridSearchCV(estimator = dt, param_grid = dt_grid, cv = skf, n_jobs = -1, verbose = 2, scoring='roc_auc')
    dt_grid.fit(X_train, y_train)
    
    dt_best_grid = dt_grid.best_estimator_
    dt_grid_accuracy = evaluate(dt_best_grid, X_train, y_train, X_test, y_test)
    print("Best Params Grid Search for DT")
    print(dt_grid.best_params_)
    
    return dt_best_grid
    
def grid_search_rf(X, y):
    print("GRID SEARCH RANDOM FOREST EXPERIMENT")
    rf_grid = {
        'bootstrap': [False],
        'max_depth': [20,30,40],
        'max_features': [9,10,11],
        'min_samples_leaf': [1,2,3],
        'min_samples_split': [4,5,6],
        'n_estimators': [1100,1200,1300]
    }
    
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    over = SMOTE(sampling_strategy=0.2, random_state = 42)
    under = RandomUnderSampler(sampling_strategy=0.4, random_state = 42)
    steps = [('o', over), ('u', under)]
    pipeline = Pipeline(steps=steps)
    X_train, y_train = pipeline.fit_resample(X_train, y_train)
    
    rf = RandomForestClassifier(random_state=42)
    rf_grid = GridSearchCV(estimator = rf, param_grid = rf_grid, cv = skf, n_jobs = -1, verbose = 2, scoring='roc_auc')
    rf_grid.fit(X_train, y_train)
    
    rf_best_grid = rf_grid.best_estimator_
    rf_grid_accuracy = evaluate(rf_best_grid, X_train, y_train, X_test, y_test)
    print("Best Params Grid Search for RF")
    print(rf_grid.best_params_)
    
    return rf_best_grid

In [20]:
dt_best_grid = grid_search_dt(X, y)

GRID SEARCH DECISION TREE EXPERIMENT
Fitting 5 folds for each of 81 candidates, totalling 405 fits
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.8959    0.9579    0.9259       665
         1.0     0.8727    0.7218    0.7901       266

    accuracy                         0.8904       931
   macro avg     0.8843    0.8398    0.8580       931
weighted avg     0.8893    0.8904    0.8871       931

FBeta Score
0.7476635514018691
Model Performance
Accuracy = 0.8904%.
AUC ROC = 0.9550%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9784    0.9411    0.9594       577
         1.0     0.3061    0.5556    0.3947        27

    accuracy                         0.9238       604
   macro avg     0.6423    0.7483    0.6771       604
weighted avg     0.9483    0.9238    0.9341       604

FBe

/opt/homebrew/Caskroom/miniforge/base/envs/.venv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
135 fits failed out of a total of 405.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
135 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/.venv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/homebrew/Caskroom/miniforge/base/envs/.venv/lib/python3.9/site-packages/sklearn/tree/_classes.py", line 937, in fit
    super().fit(
  File "/opt/homebrew/Caskroom/miniforge/base/envs/.venv/lib/python3.9/site-packages/sklearn/

In [21]:
rf_best_grid = grid_search_rf(X, y)

GRID SEARCH RANDOM FOREST EXPERIMENT
Fitting 5 folds for each of 243 candidates, totalling 1215 fits
TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.9985    1.0000    0.9992       665
         1.0     1.0000    0.9962    0.9981       266

    accuracy                         0.9989       931
   macro avg     0.9992    0.9981    0.9987       931
weighted avg     0.9989    0.9989    0.9989       931

FBeta Score
0.9969902182091799
Model Performance
Accuracy = 0.9989%.
AUC ROC = 1.0000%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9774    0.9757    0.9766       577
         1.0     0.5000    0.5185    0.5091        27

    accuracy                         0.9553       604
   macro avg     0.7387    0.7471    0.7428       604
weighted avg     0.9561    0.9553    0.9557       604

F

[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.0005, min_samples_leaf=30, min_samples_split=2; total time=   0.0s
[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.0005, min_samples_leaf=32, min_samples_split=2; total time=   0.0s
[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.0004, min_samples_leaf=30, min_samples_split=1; total time=   0.0s
[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.0004, min_samples_leaf=30, min_samples_split=1; total time=   0.0s
[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.0004, min_samples_leaf=31, min_samples_split=3; total time=   0.0s
[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.0004, min_samples_leaf=31, min_samples_split=3; total time=   0.0s
[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.000

[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.0005, min_samples_leaf=30, min_samples_split=1; total time=   0.0s
[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.0005, min_samples_leaf=30, min_samples_split=2; total time=   0.0s
[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.0005, min_samples_leaf=30, min_samples_split=3; total time=   0.0s
[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.0005, min_samples_leaf=31, min_samples_split=1; total time=   0.0s
[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.0005, min_samples_leaf=31, min_samples_split=1; total time=   0.0s
[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.0005, min_samples_leaf=31, min_samples_split=3; total time=   0.0s
[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.000

[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.0005, min_samples_leaf=30, min_samples_split=1; total time=   0.0s
[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.0005, min_samples_leaf=30, min_samples_split=3; total time=   0.0s
[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.0005, min_samples_leaf=31, min_samples_split=1; total time=   0.0s
[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.0005, min_samples_leaf=31, min_samples_split=3; total time=   0.0s
[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.0005, min_samples_leaf=32, min_samples_split=1; total time=   0.0s
[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.0005, min_samples_leaf=32, min_samples_split=2; total time=   0.0s
[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.000

[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.0005, min_samples_leaf=30, min_samples_split=2; total time=   0.0s
[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.0005, min_samples_leaf=31, min_samples_split=2; total time=   0.0s
[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.0005, min_samples_leaf=31, min_samples_split=2; total time=   0.0s
[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.0005, min_samples_leaf=31, min_samples_split=3; total time=   0.0s
[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.0005, min_samples_leaf=32, min_samples_split=2; total time=   0.0s
[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.0005, min_samples_leaf=32, min_samples_split=3; total time=   0.0s
[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.000

[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.0005, min_samples_leaf=30, min_samples_split=1; total time=   0.0s
[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.0005, min_samples_leaf=31, min_samples_split=1; total time=   0.0s
[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.0005, min_samples_leaf=31, min_samples_split=2; total time=   0.0s
[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.0005, min_samples_leaf=31, min_samples_split=3; total time=   0.0s
[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.0005, min_samples_leaf=32, min_samples_split=1; total time=   0.0s
[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.0005, min_samples_leaf=32, min_samples_split=2; total time=   0.0s
[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.000

[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.0005, min_samples_leaf=30, min_samples_split=1; total time=   0.0s
[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.0005, min_samples_leaf=30, min_samples_split=2; total time=   0.0s
[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.0005, min_samples_leaf=30, min_samples_split=3; total time=   0.0s
[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.0005, min_samples_leaf=30, min_samples_split=3; total time=   0.0s
[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.0005, min_samples_leaf=31, min_samples_split=1; total time=   0.0s
[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.0005, min_samples_leaf=32, min_samples_split=2; total time=   0.0s
[CV] END criterion=entropy, max_depth=None, max_features=14, min_impurity_decrease=0.000

Best Params Grid Search for DT
{'criterion': 'entropy', 'max_depth': None, 'max_features': 15, 'min_impurity_decrease': 0.0005, 'min_samples_leaf': 32, 'min_samples_split': 2}

Best Params Grid Search for RF
{'bootstrap': False, 'max_depth': 20, 'max_features': 10, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 1200}